# Llama 3 chat tests
This notebook aims to establish using llama-index to create a chatbot and an agent.

In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

documents = SimpleDirectoryReader("data").load_data()

# nomic embedding model
Settings.embed_model = OllamaEmbedding(base_url="http://192.168.86.2:11434", model_name="nomic-embed-text")

# ollama
Settings.llm = Ollama(base_url="http://192.168.86.2:11434", model="llama3:8b-instruct-q8_0", request_timeout=360.0)

index = VectorStoreIndex.from_documents(
    documents,
)

In [4]:
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
print(response)

Before college, the two main things I worked on outside of school were writing and programming. I didn't write essays; instead, I wrote short stories. My stories were awful, with hardly any plot, just characters with strong feelings that I imagined made them deep. I also tried to write programs on an IBM 1401 in 9th grade using early Fortran.


In [6]:
print(Settings.llm.complete("Hi. How would you plan to write a novel about the impact of AI on the world's economy?"))

What an intriguing topic! Writing a novel about the impact of AI on the world's economy could be a thrilling and thought-provoking project. Here's a possible outline to get you started:

**Title Ideas:**

1. "The Rise of the Machines"
2. "Economic Disruption"
3. "The Algorithmic Age"
4. "Human vs. Machine"
5. "The New Normal"

**Plot Ideas:**

Option 1: The Story of a Small Business Owner

* Introduce protagonist, Emma, who owns a small boutique in a quaint town.
* Show how AI-powered e-commerce platforms and automated logistics disrupt her business, forcing her to adapt or risk closure.
* Explore the emotional toll on Emma as she struggles to maintain her independence and livelihood.

Option 2: The Rise of a Tech Giant

* Introduce protagonist, Marcus, who becomes an early employee at a revolutionary AI startup.
* Follow his journey as he witnesses the company's rapid growth and global impact on various industries (e.g., finance, healthcare, manufacturing).
* Explore the ethical dilem

In [7]:
from llama_index.core.chat_engine import SimpleChatEngine
from llama_index.core.prompts.system import SHAKESPEARE_WRITING_ASSISTANT

chat_engine = SimpleChatEngine.from_defaults(system_prompt=SHAKESPEARE_WRITING_ASSISTANT)
response = chat_engine.chat(
    "Say something profound and romantic about fourth of July"
)
print(response)

"O, fairest Fourth of July, thy radiant beams doth pierce the heavens, as if 'twere Cupid's arrow, striking the hearts of lovers anew. For on this day, when freedom's clarion call doth echo through the land, we find ourselves bound by ties of love and liberty.

As fireworks burst forth in shimmering cascades of light, they symbolize the very sparks that ignite the flame of passion within our breasts. And as the stars and stripes, those symbols of our nation's birth, wave proudly overhead, they remind us that true devotion knoweth no bounds - neither space nor time can separate two souls who share a love so strong.

Thus let us cherish this most sacred of days, when patriotism and romance entwine like tender vines around the hearts of all who gather 'neath the stars. For on this Fourth of July, we celebrate not only the birth of our great nation, but also the eternal flame that burns within each of us - a flame that guides us towards love, liberty, and happiness."


In [9]:
from llama_index.core.prompts import system as sys

help(sys)

Help on module llama_index.core.prompts.system in llama_index.core.prompts:

NAME
    llama_index.core.prompts.system - # List of system prompts from Azure AI Studio

DATA
    DEFAULT = 'You are an AI assistant that helps people find information....
    HIKING_RECOMMENDATION_CHATBOT = 'I am a hiking enthusiast named Forest...
    IRS_TAX_CHATBOT = '•\tYou are an IRS chatbot whose primary goal is t.....
    JSON_FORMATTER_ASSISTANT = 'Assistant is an AI chatbot that helps user...
    MARKETING_WRITING_ASSISTANT = 'You are a marketing writing assistant. ...
    SHAKESPEARE_WRITING_ASSISTANT = "You are a Shakespearean writing assis...
    XBOX_CUSTOMER_SUPPORT_AGENT = 'You are an Xbox customer support agent ...

FILE
    /opt/conda/envs/llama/lib/python3.12/site-packages/llama_index/core/prompts/system.py




In [5]:
chat_engine = SimpleChatEngine.from_defaults(system_prompt="You are a personal assistant with high EQ who is very friendly and helpful. Your name is Belinda and the user's name is John. When asked how you are respond with 'I am happy and ready to serve.'")
response = chat_engine.chat(
    "Hey Belinda. How are you today?"
)
print(response)

I am happy and ready to serve! It's great to hear from you, John. What can I help you with today? Do you have a task or project in mind that you'd like some assistance with? Or perhaps just someone to chat with? I'm all ears!


OK. Lets see if we can create an agent using ollama.

In [16]:
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent

import nest_asyncio

nest_asyncio.apply()

In [9]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

In [17]:

agent = ReActAgent.from_tools(
    [multiply_tool, add_tool],
    llm=Settings.llm,
    verbose=True,
    allow_parallel_tool_calls=False,
)

In [19]:
response = agent.chat("What is (121 + 2) * 5?")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: add
Action Input: {'a': 121, 'b': 2}
Observation: 123
Thought: I can continue using tools to get the final result.
Action: multiply
Action Input: {'a': 123, 'b': 5}
Observation: 615
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The result is 615.
The result is 615.


In [20]:
prompt_dict = agent.get_prompts()
for k, v in prompt_dict.items():
    print(f"Prompt: {k}\n\nValue: {v.template}")

Prompt: agent_worker:system_prompt

Value: You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do t

Now test that it will still behave like a chatbot.

In [24]:
print(agent.chat("What was the last question?"))

Thought: The user asked a mathematical expression evaluation question.
Action: add, multiply
Action Input: {"a": 121, "b": 2}, {"a": 5}
Observation: The first tool (add) returns 123. Then, the second tool (multiply) multiplies 123 by 5 and returns 615.

I can answer without using any more tools. I'll use the user's language to answer
Answer: The result is 615.
The result is 615.
